# Configure Preprocessing Params

This notebook should be used to set up preprocessing params.
Cells marked with <font color='red'>SET PARAMETERS</font> contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## Imports

In [4]:
from pathlib import Path

import yaml
from microfilm.microplot import Microimage
import matplotlib.pyplot as plt

from lib.shared.configuration_utils import (
    CONFIG_FILE_HEADER,
    create_samples_df,
    create_micropanel,
)
from lib.preprocess.preprocess import extract_tile_metadata, nd2_to_tiff
from lib.preprocess.file_utils import get_sample_fps

## <font color='red'>SET PARAMETERS</font>

### Fixed parameters for preprocessing

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing*.
- `ROOT_FP`: Path to root of Brieflow output directory*.

*Note: Paths can be absolute or relative to where workflows are run from.

In [ ]:
CONFIG_FILE_PATH = "config/config.yml"
ROOT_FP = "analysis_root"

Path(CONFIG_FILE_PATH).parent.mkdir(parents=True, exist_ok=True)
Path(ROOT_FP).mkdir(parents=True, exist_ok=True)

## <font color='red'>SET PARAMETERS</font>

### Paths to dataframes with sample information

- `SBS_SAMPLES_DF_FP`/`PHENOTYPE_SAMPLES_DF_FP`: Path to dataframe where SBS/phenotype samples location and metadata will be stored.
- `SBS_COMBO_DF_FP`/`PHENOTYPE_COMBO_DF_FP`: Path to dataframe where SBS/phenotype sample metadata combinations will be stored.
- `SBS_IMAGES_DIR_FP`/`PHENOTYPE_IMAGES_DIR_FP`: Path to directories with SBS/phenotype sample nd2 files. Set to `None` to ignore SBS/phenotype testing in this notebook.

### Pattern configurations for metadata extraction

#### SBS Configuration
- `SBS_PATH_PATTERN`: Regex pattern to match directory structure of SBS files
- `SBS_PATH_METADATA`: List of metadata to extract from file path
    - Should include at least `"plate", "well", "tile, "cycle""` to extract SBS processing information
- `SBS_METADATA_ORDER_TYPE`: Metadata order will be used to organize the file paths dataframe. Metadata types will be used to convert parsed information.

#### Phenotype Configuration
- `PHENOTYPE_PATH_PATTERN`: Regex pattern to match directory structure of phenotype files  
- `PHENOTYPE_PATH_METADATA`: List of metadata to extract from file path
    - Should include at least `"plate", "well", "tile"` to extract phentoype processing information
- `PHENOTYPE_METADATA_ORDER_TYPE`: Metadata order will be used to organize the file paths dataframe. Metadata types will be used to convert parsed information.

*Notes:*
- Paths can be absolute or relative to where workflows are run from
- Each pattern (path and sample) should have the same number of capture groups as pieces of metadata listed
- Metadata lists should be ordered to match the capture groups in their corresponding regex pattern
- For both path and sample patterns, numeric values (like cycle numbers) will automatically be converted to integers
- For Brieflow to run effectively, each sample fil epath should have an associated plate/well. For single plate/well screens manually add a plate/well to the file path dataframe.

In [6]:
# paths to sample dataframes
SBS_SAMPLES_DF_FP = "config/sbs_samples.tsv"
PHENOTYPE_SAMPLES_DF_FP = "config/phenotype_samples.tsv"
# paths to combo dataframes
SBS_COMBO_DF_FP = "config/sbs_combo.tsv"
PHENOTYPE_COMBO_DF_FP = "config/phenotype_combo.tsv"

# Directory, pattern, and metadata for SBS sample nd2 files
SBS_IMAGES_DIR_FP = Path("/lab/cheesegrater_ops/nebo/")
SBS_SAMPLE_PATTERN = r"plate_(\d+)/input_sbs/c(\d+)/(\d+x)/P\d+_Wells-([A-Z]\d+)_Points-(\d+)__Channel_.*\.nd2"
SBS_PATH_METADATA = ["plate", "cycle", "magnification", "well", "tile"]
SBS_METADATA_ORDER_TYPE = {
    "plate": int,
    "cycle": int,
    "magnification": str,
    "well": str,
    "tile": int,
}

# Directory, pattern, and metadata for phenotype sample nd2 files
PHENOTYPE_IMAGES_DIR_FP = Path("/lab/cheesegrater_ops/nebo/")
PHENOTYPE_SAMPLE_PATTERN = r"plate_(\d+)/input_ph/round_(\d+)/\d+_\d+_\d+/P\d+_Wells-([A-Z]\d+)_Points-(\d+)__Channel_.*\.nd2"
PHENOTYPE_PATH_METADATA = ["plate", "round", "well", "tile"]
PHENOTYPE_METADATA_ORDER_TYPE = {
    "plate": int,
    "well": str,
    "tile": int,
    "round": int,
}

You must supply a working regex to the `SBS_PATH_PATTERN` and `PHENOTYPE_PATH_PATTERN` variables. If you don't have experience with regex, you can use the following LLM prompt to generate the patterns.

*Enter into a basic LLM chatbot*: 

Given ND2 filenames from your experiment, generate regex patterns to extract metadata. Return only the regex patterns with no explanation.

Example sbs filenames: **[ENTER YOUR EXAMPLE SBS FILES HERE WITH ANY UPSTREAM FOLDER STRUCTURE THAT IS RELEVANT TO THE METADATA]**

Example phenotype filenames: **[ENTER YOUR EXAMPLE PHENOTYPE FILES HERE WITH ANY UPSTREAM FOLDER STRUCTURE THAT IS RELEVANT TO THE METADATA]**

Required regex patterns (return these exact variable assignments):
```python
SBS_PATH_PATTERN = r"..."      # To match file path structure
PHENOTYPE_PATH_PATTERN = r"..." # To match file path structure
```

The patterns should extract:
1. SBS pattern:
     - Plate number (after "plate_")
     - Well ID (e.g., "A1", "B2")
     - Tile number (after "Points-")
     - Cycle number (after "/c")
2. PHENOTYPE pattern:
     - Plate number (after "plate_")
     - Well ID (e.g., "A1", "B2")
     - Tile number (after "Points-")

Also provide the corresponding metadata lists and variable types:
```python
SBS_PATH_METADATA = ["plate", "cycle", "well", "tile"]
PHENOTYPE_PATH_METADATA = ["plate", "well", "tile"]
SBS_METADATA_ORDER_TYPE = {"plate": int, "well": str, "tile": int, "cycle": int}
PHENOTYPE_METADATA_ORDER_TYPE = {"plate": int, "well": str, "tile": int}
```

Example patterns for reference:
```python
SBS_PATH_PATTERN = r"plate_(\d+)/c(\d+)/.*_Wells-([A-Z]\d+)_Points-(\d+)__.*"
PHENOTYPE_PATH_PATTERN = r"P(\d+)_Pheno_20x_Wells-([A-Z]\d+)_Points-(\d+)__.*"

SBS_PATH_METADATA = ["plate", "cycle", "well", "tile"]
PHENOTYPE_PATH_METADATA = ["plate", "well", "tile"]

SBS_METADATA_ORDER_TYPE = {"plate": int, "well": str, "tile": int, "cycle": int}
PHENOTYPE_METADATA_ORDER_TYPE = {"plate": int, "well": str, "tile": int}
```

## Create Sample DFs

In [7]:
# Create sample dataframe for sbs
sbs_samples = create_samples_df(
    SBS_IMAGES_DIR_FP, SBS_SAMPLE_PATTERN, SBS_PATH_METADATA, SBS_METADATA_ORDER_TYPE
)

# Filter sbs_samples to only keep 10x tiles 100–130 from well A3
sbs_samples = sbs_samples[
    (sbs_samples["well"] == "A3")
    & (sbs_samples["magnification"] == "10x")
    & (sbs_samples["tile"].astype(int) >= 100)
    & (sbs_samples["tile"].astype(int) <= 130)
]

sbs_wildcard_combos = sbs_samples[SBS_PATH_METADATA].drop_duplicates().astype(str)

# Create sample dataframe for phenotype
phenotype_samples = create_samples_df(
    PHENOTYPE_IMAGES_DIR_FP,
    PHENOTYPE_SAMPLE_PATTERN,
    PHENOTYPE_PATH_METADATA,
    PHENOTYPE_METADATA_ORDER_TYPE,
)
# # Filter out well B1
# phenotype_samples = phenotype_samples[
#     phenotype_samples["well"].str.contains(r"^(?!B1).*")
# ]
# Filter phenotype tiles to only keep tiles 
phenotype_samples = phenotype_samples[
    (phenotype_samples["well"] == "A3")
    & (phenotype_samples["tile"].astype(int) >= 4020)
    & (phenotype_samples["tile"].astype(int) <= 4040)
]

phenotype_wildcard_combos = (
    phenotype_samples[PHENOTYPE_PATH_METADATA].drop_duplicates().astype(str)
)

# Save and display SBS sample dataframe
sbs_samples.to_csv(SBS_SAMPLES_DF_FP, sep="\t", index=False)
print("SBS samples:")
display(sbs_samples)
sbs_wildcard_combos.to_csv(SBS_COMBO_DF_FP, sep="\t", index=False)
print("SBS wildcard combos:")
display(sbs_wildcard_combos)

# Save and display Phenotype sample dataframe
phenotype_samples.to_csv(PHENOTYPE_SAMPLES_DF_FP, sep="\t", index=False)
print("Phenotype samples:")
display(phenotype_samples)
phenotype_wildcard_combos.to_csv(PHENOTYPE_COMBO_DF_FP, sep="\t", index=False)
print("Phenotype wildcard combos:")
display(phenotype_wildcard_combos)

SBS samples:


,sample_fp,plate,cycle,magnification,well,tile
100,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,100
101,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,101
102,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,102
103,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,103
104,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,104
105,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,105
106,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,106
107,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,107
108,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,108
109,/lab/cheesegrater_ops/nebo/plate_1/input_sbs/c...,1,1,10x,A3,109


SBS wildcard combos:


,plate,cycle,magnification,well,tile
100,1,1,10x,A3,100
101,1,1,10x,A3,101
102,1,1,10x,A3,102
103,1,1,10x,A3,103
104,1,1,10x,A3,104
105,1,1,10x,A3,105
106,1,1,10x,A3,106
107,1,1,10x,A3,107
108,1,1,10x,A3,108
109,1,1,10x,A3,109


Phenotype samples:


,sample_fp,plate,well,tile,round
12060,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4020,1
12061,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4020,2
12062,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4020,3
12063,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4021,1
12064,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4021,2
...,...,...,...,...,...
12118,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4039,2
12119,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4039,3
12120,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4040,1
12121,/lab/cheesegrater_ops/nebo/plate_1/input_ph/ro...,1,A3,4040,2


Phenotype wildcard combos:


,plate,round,well,tile
12060,1,1,A3,4020
12061,1,2,A3,4020
12062,1,3,A3,4020
12063,1,1,A3,4021
12064,1,2,A3,4021
...,...,...,...,...
12118,1,2,A3,4039
12119,1,3,A3,4039
12120,1,1,A3,4040
12121,1,2,A3,4040


## Test Metadata Extraction

In [8]:
# if SBS_IMAGES_DIR_FP is not None:
#     # Extract metadata from test sbs sample
#     test_sbs_metadata = extract_tile_metadata(
#         sbs_samples["sample_fp"][0],
#         sbs_samples["plate"][0],
#         sbs_samples["well"][0],
#         sbs_samples["tile"][0],
#         verbose=True,
#     )
#     print("SBS test metadata:")
#     display(test_sbs_metadata)

# if PHENOTYPE_IMAGES_DIR_FP is not None:
#     # Extract metadata from test phenotype sample
#     test_phenotype_metadata = extract_tile_metadata(
#         phenotype_samples["sample_fp"][0],
#         phenotype_samples["plate"][0],
#         phenotype_samples["well"][0],
#         phenotype_samples["tile"][0],
#         verbose=True,
#     )
#     print("Phenotype test metadata:")
#     display(test_phenotype_metadata)

## <font color='red'>SET PARAMETERS</font>

### ND2 conversion

- `SBS_CHANNEL_ORDER`/`PHENOTYPE_CHANNEL_ORDER`: Manually set channel order _if ND2 images are acquired as single channels, or there are multiple files for each tile (e.g. multiple rounds of phenotype images). Should be `None` if multichannel image files are acquired. The extracted channel names must match the values that will be displayed in the samples DataFrame channel column (e.g., `["DAPI", "GFP", "CY3", "CY5", "AF750"]`).
- `PHENOTYPE_ROUND_ORDER`: List of round numbers specifying the order in which to process phenotype image rounds. Should be `None` if there is only one round of phenotyping. For multiple rounds, specify the round numbers in the desired order (e.g., `[1, 2, 3]`). The round numbers must match the values in the samples DataFrame round column.

**Note** For single-channel files, each file must contain a channel identifier that your regex can extract. For multichannel files, set the channel patterns to `None`. Metadata extraction is only performed on the first channel dimension for each tile. Please ensure the Dapi channel is displayed first!

- `SBS_CHANNEL_ORDER_FLIP`/`PHENOTYPE_CHANNEL_ORDER_FLIP`: Whether or not to flip channel order when converting ND2->tiff. Should be `False` if channels are in a standard order (with Dapi first), or `True` if channels are reversed. This will only occur for multichannel ND2 files, for each individual ND2 file. Setting the channel order for single channel files is done by setting `SBS_CHANNEL_ORDER`/`PHENOTYPE_CHANNEL_ORDER` previously.

**Note** Channel order can be checked with the test conversions below. Please ensure the Dapi channel is displayed first!

In [9]:
SBS_CHANNEL_ORDER = None
PHENOTYPE_CHANNEL_ORDER = None
PHENOTYPE_ROUND_ORDER = [1, 2, 3]

In [10]:
SBS_CHANNEL_ORDER_FLIP = None
PHENOTYPE_CHANNEL_ORDER_FLIP = False

In [11]:
#Set parameters for ND2 to TIFF conversion
Z_HANDLING="best_focus"
# If using best_slice
BEST_FOCUS_CHANNEL=[3, 1, 2]

## Test ND2 Conversion

### Test SBS conversion

In [12]:
# if SBS_IMAGES_DIR_FP is not None:
#     # Get test sbs sample
#     sbs_sample = get_sample_fps(
#         sbs_samples,
#         plate=sbs_samples["plate"][0],
#         well=sbs_samples["well"][0],
#         tile=sbs_samples["tile"][200],
#         cycle=sbs_samples["cycle"][0],
#         channel_order=SBS_CHANNEL_ORDER,
#     )

#     # Convert test sbs sample to tiff
#     sbs_image = nd2_to_tiff(sbs_sample, SBS_CHANNEL_ORDER_FLIP, verbose=True)
#     # Create micropanel to display converted samples
#     print("Converted SBS test sample:")
#     sbs_microimages = [Microimage(image) for image in sbs_image]
#     sbs_panel = create_micropanel(sbs_microimages, add_channel_label=False)
#     plt.show()

### Test phenotype conversion

In [13]:
# if PHENOTYPE_IMAGES_DIR_FP is not None:
#     # Get test phenotype sample
#     phenotype_sample = get_sample_fps(
#         phenotype_samples,
#         plate=phenotype_samples["plate"][0],
#         well=phenotype_samples["well"][0],
#         tile=phenotype_samples["tile"][405],
#         round_order=PHENOTYPE_ROUND_ORDER,
#         channel_order=PHENOTYPE_CHANNEL_ORDER,
#     )
#     # Convert test phenotype sample to tiff
#     phenotype_image = nd2_to_tiff(
#         phenotype_sample,
#         PHENOTYPE_CHANNEL_ORDER_FLIP,
#         best_focus_channel=BEST_FOCUS_CHANNEL, 
#         verbose=False,
#         z_handling=Z_HANDLING
#     )

#     # Create micropanel to display converted samples
#     print("Converted phenotype test sample:")
#     phenotype_microimages = [Microimage(image) for image in phenotype_image]
#     phenotype_panel = create_micropanel(phenotype_microimages, add_channel_label=False)
#     plt.show()

In [14]:
SAMPLE_FRACTION = 1

## Create config file with params

In [15]:
# Create empty config variable
config = {}

# Add all section
config["all"] = {
    "root_fp": ROOT_FP,
}

# Add preprocess section
config["preprocess"] = {
    "sbs_samples_fp": SBS_SAMPLES_DF_FP,
    "sbs_combo_fp": SBS_COMBO_DF_FP,
    "phenotype_samples_fp": PHENOTYPE_SAMPLES_DF_FP,
    "phenotype_combo_fp": PHENOTYPE_COMBO_DF_FP,
    "sbs_channel_order": SBS_CHANNEL_ORDER,
    "phenotype_channel_order": PHENOTYPE_CHANNEL_ORDER,
    "phenotype_round_order": PHENOTYPE_ROUND_ORDER,
    "sbs_channel_order_flip": SBS_CHANNEL_ORDER_FLIP,
    "phenotype_channel_order_flip": PHENOTYPE_CHANNEL_ORDER_FLIP,
    "sample_fraction":SAMPLE_FRACTION,
    "phenotype_z_handling": Z_HANDLING,
    "phenotype_best_focus_channel": BEST_FOCUS_CHANNEL,
    "sbs_z_handling": "no_z",
    "sbs_best_focus_channel": [0],
}

# Write the updated configuration back with markdown-style comments
with open(CONFIG_FILE_PATH, "w") as config_file:
    # Write the introductory markdown-style comments
    config_file.write(CONFIG_FILE_HEADER)

    # Dump the updated YAML structure, keeping markdown comments for sections
    yaml.dump(config, config_file, default_flow_style=False)